![](https://f4.bcbits.com/img/a1460053395_10.jpg)

<div class="alert alert-block alert-warning">

### In this Notebook, we will look at(Theory & Code),

1. Various components of time-series data such as trend, seasonality, cyclical component, and random component. 

2. Moving Average, Exponenial Smoothing, Auto-Regressive (AR), Moving Average (MA), and Auto-Regressive Integrated Moving Average (ARIMA) models.

3. Building forecasting models in Python.

# CONTENTS

1. [What is VMAP & ITC](#vitc)
2. [Components of Time series](#cmp)
3. [Models](#tc)

   3.1 [Moving Average](#mv)
   3.2 [Exponential Smoothing](#exps)

4. [Decomposing Time Series](dts)

5. [ARIMA](#arima)

   5.1 [AR Model](#ar)
   5.2 [MA Model](#ma)
   5.3 [ARMA](#arma)
   5.4 [ARIMA](#arimaa)

6. [Summary](#summ)

In [ ]:
import numpy as np 
import pandas as pd 
import os
from fbprophet import Prophet
from matplotlib import pyplot as plt
import plotly.express as px
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

#### Been noticing people go gaga over ITC stock, especially on twitter.

### So, lets look into it

<a id="vitc"></a>
![](https://lh3.googleusercontent.com/proxy/YINfgR9lpvMaA7OkTVtML679-6em_cIsiEa5FQ8mXUFWnHiunYrda0ObEQ7UaUnarDTHa22-LRmvB4-gV82l04q3-YpVf9vtj2WB7Sa6NbR27vSVIg)

<div class="alert alert-block alert-info">

ITC Limited is an Indian multinational conglomerate company headquartered in Kolkata, West Bengal.

Established in 1910 as the Imperial Tobacco Company of India Limited, the company was renamed as the India Tobacco Company Limited in 1970 and later to I.T.C. The Company now stands rechristened to ITC Limited, where 'ITC' is today no longer an acronym or an initialised form.

It employs over 30,000 people at more than 60 locations across India and is part of Forbes 2000 list.

Source - [Here](https://en.wikipedia.org/wiki/ITC_Limited)

In [ ]:
itc = pd.read_csv("/kaggle/input/nifty50-stock-market-data/ITC.csv")
itc.set_index("Date", drop=False, inplace=True)
itc.head()

In [ ]:
print(f'There are {itc.shape[0]} rows and {itc.shape[1]} columns')

<div class="alert alert-block alert-warning">
#### Across the following sections, we will consider VMAP as the target variable and look at the basic approaches to forecast that, but first lets look at how it has performed over the years.

#### So What is VWAP? 

#### The volume weighted average price (VWAP) is a trading benchmark used by traders that gives the average price a security has traded at throughout the day, based on both volume and price. It is important because it provides traders with insight into both the trend and value of a security. You can read more [here](https://en.wikipedia.org/wiki/Volume-weighted_average_price)

In [ ]:
itcv = itc.VWAP.reset_index()
fig = px.line(itcv, x="Date", y="VWAP", title='VMAP of ITC over the years')
fig.show()

### There is a sharp decline in 2005. Lets see what happened there.

In [ ]:
df_vwap = itc[['Date','VWAP']]
df_vwap['Date'] = df_vwap['Date'].apply(pd.to_datetime)
df_vwap.set_index("Date", inplace = True)
df_vwap.head()

In [ ]:
vwap_2005 = df_vwap['2005':'2006']
vwap_2005 = vwap_2005.reset_index()
fig = px.line(vwap_2005, x="Date", y="VWAP", title='VMAP of ITC over the years')
fig.show()

<div class="alert alert-block alert-info">

### Something happened between Sep 20 and Sep 22nd, 2005.

### So, The Face value had dropped from 10 to 1, and guess the date? Yep, Sep 21, 2005. More on it [here](https://www.moneycontrol.com/company-facts/itc/splits/itc)

<div class="alert alert-block alert-warning">

<a id="cmp"></a>

### Components of Time-Series Data

<div class="alert alert-block alert-info">

- The time-series data Y(t) is a random variable, usually collected at regular time intervals and in chronological order.
- If the time-series data contains observations of just a single variable (such as demand of a product at time t), 
then it is termed as univariate time-series data. 

- If the data consists of more than one variable, 
for example, demand for a product at time t, price at time t, amount of money spent by the company on promotion at time t, 
competitors’ price at time t, etc., then it is called multivariate timeseries data. 

A time-series data can be broken into the four following components:

- Trend Component - Trend is the consistent long-term upward or downward movement of the data. 
- Seasonal Component (St): Seasonal component (measured using seasonality index) is the repetitive upward or downward movement (or fluctuations) from the trend that occurs within a calendar year at fixed intervals (i.e., time between seasons is fixed) such as seasons, quarters, months, days of the week, etc. The upward or downward fluctuation may be caused due to festivals, customs within a society,  school holidays, business practices within the market such as “end of season sale”, and so on.
- Cyclical Component (Ct): Cyclical component is fluctuation around the trend line at random interval 
   (i.e., the time between cycles is random) that happens due to macro-economic changes such as recession, unemployment,
   etc. Cyclical fluctuations have repetitive patterns with time between repetitions of more than a year.
   Whereas in the case of seasonality, the fluctuations are observed within a calendar year and are driven by factors such as     festivals and customs that exist in a society. A major difference between seasonal fluctuation and cyclical fluctuation is that seasonal fluctuation occurs at fixed period within a calendar year, whereas cyclical fluctuations have random time between fluctuations. That is, the periodicity of seasonal fluctuations is constant, whereas the periodicity of cyclical fluctuations is not constant.
- Irregular Component (It): Irregular component is the white noise or random uncorrelated changes that follow a normal distribution with mean value of 0 and constant variance.    

<div class="alert alert-block alert-warning">

<a id="tc"></a>

### Forecasting Techniques

<div class="alert alert-block alert-info">

There are several techniques, the famous ones being;
    - Moving average, 
    - Exponential smoothing, and 
    - AutoRegressive Integrated Moving Average (ARIMA)
    
Moving average and exponential smoothing predict the future value of a time-series data as a function of past observations. 

The regression-based models such as auto-regressive (AR), auto-regressive and moving average (ARMA), 
auto-regressive integrated moving average (ARIMA) use more sophisticated regression models. 

<div class="alert alert-block alert-info">

<a id="mv"></a>

### Forecasting using Moving Average

<div class="alert alert-block alert-warning">

Moving average is one of the simplest forecasting techniques which forecasts the future value of a timeseries data using average (or weighted average) of the past N observations. 

Forecasted value for time t +1 using the simple moving average is given by
        
![](https://github.com/rakash/images1/blob/master/fma.png?raw=true)
            
Pandas has a function rolling() which can be used with an aggregate function like mean() for calculating moving average for a time window. 

For example, to calculate 12 month’s moving average using last 12 months’ data starting from last month (previous period), rolling() will take a parameter window, which is set to 12 to indicate moving average of 12-months data, and then use Pandas’ shift() function, which takes parameter 1 to specify that the 12-months data should start from last month. 

shift(1) means calculating moving average for the specified window period starting from previous observation (in this case last month).


Here, we will take data from 2018 April to 2019 April, to forecast VWAP of May 2019 using Moving Average

In [ ]:
itcv = itc.VWAP.reset_index()
fig = px.line(itcv, x="Date", y="VWAP", title='VMAP of ITC over the years')
fig.show()

In [ ]:
vwap_itc_2020 = df_vwap['2019-04':'2020-05']
vwap_itc_2020['mavg_12'] = vwap_itc_2020['VWAP'].rolling(window = 12).mean().shift(1)
pd.set_option('display.float_format', lambda x: '%.2f'% x)
mayd = vwap_itc_2020.tail(19)
mayd = mayd.reset_index()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=mayd['Date'],
    y=mayd['VWAP'],
    name="Actual VWAP"
))

fig.add_trace(go.Scatter(
    x=mayd['Date'],
    y=mayd['mavg_12'],
    name="Moving Average"
))

fig.update_layout(
    title="VWAP - Actual vs Moving Average of ITC for the Month of May 2020",
    xaxis_title="Date",
    yaxis_title="VWAP",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)

fig.show()

<div class="alert alert-block alert-info">

### Calculating Forecast Accuracy


Root mean square error (RMSE) and mean absolute percentage error (MAPE) are the two most popular accuracy measures of forecasting. We will be discussing these measures in this section.

<div class="alert alert-block alert-warning">

### Mean Absolute Percentage Error


Mean absolute percentage error (MAPE) is the average of absolute percentage error.
Assume that the validation data has n observations and forecasting is carried out on these n observations. 
The mean absolute percentage error is given by

![](https://github.com/rakash/images1/blob/master/mape.png?raw=true)

In [ ]:
def get_mape(actual, predicted):
    y_true, y_pred = np.array(actual), np.array(predicted)
    return np.round( np.mean(np.abs((actual - predicted)/ actual)) * 100, 2)

In [ ]:
get_mape(mayd['VWAP'].values, mayd['mavg_12'].values)

#### The MAPE in this case is 7.21. So, forecasting using moving average gives us a MAPE of 7.2%. I would say its a decent number. 

<div class="alert alert-block alert-warning">

### Root mean square error (RMSE)

is the square root of average of squared error calculated over the validation dataset, and is the standard deviation of the errors for unbiased estimator.

RMSE is given by

![](https://github.com/rakash/images1/blob/master/rmse.png?raw=true)


Lower RMSE implies better prediction. However, it depends on the scale of the time-series data.

MSE (Mean Squared Error) can be calculated using mean_squared_error() method in sklearn.metrics.

We can pass MSE value to np.sqrt() to calculate RMSE.

In [ ]:
from sklearn.metrics import mean_squared_error 
np.sqrt(mean_squared_error(mayd['VWAP'].values, mayd['mavg_12'].values))

<div class="alert alert-block alert-info">

<a id="exps"></a>

### Forecasting using Exponential Smoothing

<div class="alert alert-block alert-warning">

One of the drawbacks of the simple moving average technique is that it gives equal weight to all the previous observations used in forecasting the future value. Exponential smoothing technique (also known as simple exponential smoothing; SES) assigns differential weights to past observations;

Equation:

![](https://github.com/rakash/images1/blob/master/exp.png?raw=true)


where α is called the smoothing constant, and its value lies between 0 and 1. Ft+1 is the forecasted value at time t + 1 using actual value Yt at time t and forecasted values Ft of time t. But the model applies differential weights to both the inputs using smoothing constant α.


The ewm() method in Pandas provides the features for computing the exponential moving average taking alpha as a parameter.

In [ ]:
vwap_itc_2020['ewm'] = vwap_itc_2020['VWAP'].ewm(alpha = 0.2 ).mean()
pd.set_option('display.float_format', lambda x: '%.2f'% x)
emayd = vwap_itc_2020.tail(19)
emayd = emayd.reset_index()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=emayd['Date'],
    y=emayd['VWAP'],
    name="Actual VWAP"
))

fig.add_trace(go.Scatter(
    x=emayd['Date'],
    y=emayd['ewm'],
    name="Exponential Smoothing"
))

fig.update_layout(
    title="VWAP - Actual vs Exponential Smoothing of ITC for the Month of May 2020",
    xaxis_title="Date",
    yaxis_title="VWAP",
    font=dict(
        family="Courier New, monospace",
        size=8,
        color="#7f7f7f"
    )
)

fig.show()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=emayd['Date'],
    y=emayd['VWAP'],
    name="Actual VWAP"
))


fig.add_trace(go.Scatter(
    x=emayd['Date'],
    y=emayd['mavg_12'],
    name="Moving Average"
))

fig.add_trace(go.Scatter(
    x=emayd['Date'],
    y=emayd['ewm'],
    name="Exponential Smoothing"
))

fig.update_layout(
    title="VWAP - Actual vs vs Moving Average vs Exponential Smoothing of ITC for the Month of May 2020",
    xaxis_title="Date",
    yaxis_title="VWAP",
    font=dict(
        family="Courier New, monospace",
        size=8,
        color="#7f7f7f"
    )
)

fig.show()

#### IMPORTANT: Moving average and Exponential smoothing assume a fairly steady time-series data with no significant trend, seasonal or cyclical components, that is, the data is stationary.

#### However, many dataset will have trend and seasonality.

<div class="alert alert-block alert-info">

<a id="dts"></a>

### DECOMPOSING TIME SERIES

<div class="alert alert-block alert-warning">

The time-series data can be modelled as addition or product of trend, seasonality, cyclical, and irregular components.
The additive time-series model is given by 

        Yt = Tt + St + Ct + It

The additive models assume that the seasonal and cyclical components are independent of the trend component.
Additive models are not very common, since in many cases the seasonal component may not be independent of the trend. 

A small example:

The seasonality effect on sales during festival times like Diwali does not result in constant increase in sales over the years. For example, the increase in sales of cars during festival season is not just 100 units every year. The seasonality effect has a multiplicative effect on sales based on the trend over the years like 10% additional units based on the trend in the current year. So, in many cases the seasonality effect is multiplied with the trend and not just added as in additive model. 

The multiplicative time-series model is given by

        Yt = Tt × St × Ct × It
        
 
 
For decomposing a time-series data, we can leverage the following libraries:
1. statsmodel.tsa provides various features for time-series analysis. 
2. seasonal_decompose() in statsmodel.tsa.seasonal decomposes a time series into trend, seasonal, and residuals. It takes frequency parameters; for example, the frequency is 12 for monthly data.

In [ ]:
vwap_itc_20201 = vwap_itc_2020.reset_index()
from statsmodels.tsa.seasonal import seasonal_decompose

# Multiplicative Decomposition 
result_mul = seasonal_decompose(vwap_itc_2020['VWAP'], model='multiplicative', extrapolate_trend = 'freq', freq=12)

# Additive Decomposition
result_add = seasonal_decompose(vwap_itc_2020['VWAP'], model='additive', extrapolate_trend = 'freq', freq=12)

# Plot
plt.rcParams.update({'figure.figsize': (10,10)})

### Multiplicative Decompose

In [ ]:
result_mul.plot().suptitle('Multiplicative Decompose', fontsize=15)

### Additive Decompose

In [ ]:
result_add.plot().suptitle('Additive Decompose', fontsize=15)

<div class="alert alert-block alert-info">

### Extracting only the seasonal and trend component for the last 1 year

In [ ]:
vwap_itc_2020['mul_seasonal'] = result_mul.seasonal
vwap_itc_2020['mul_trend'] = result_mul.trend

vwap_itc_2020['add_seasonal'] = result_add.seasonal
vwap_itc_2020['add_trend'] = result_add.trend

<div class="alert alert-block alert-info">

### Visualizing Trend and Seasonal Component for the last year of ITC VWAP

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

vwap_itc_2020d = vwap_itc_2020.reset_index()

fig.add_trace(go.Scatter(
    x=vwap_itc_2020d['Date'],
    y=vwap_itc_2020d['mul_seasonal'],
    name="Multiplicative Seasonal Component"
))

fig.add_trace(go.Scatter(
    x=vwap_itc_2020d['Date'],
    y=vwap_itc_2020d['add_seasonal'],
    name="Additive Seasonal Component"
))

fig.update_layout(
    title="Seasonal Component",
    xaxis_title="Date",
    yaxis_title="VWAP",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)

fig.show()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

vwap_itc_2020d = vwap_itc_2020.reset_index()

fig.add_trace(go.Scatter(
    x=vwap_itc_2020d['Date'],
    y=vwap_itc_2020d['mul_trend'],
    name="Multiplicative Trend"
))

fig.add_trace(go.Scatter(
    x=vwap_itc_2020d['Date'],
    y=vwap_itc_2020d['add_trend'],
    name="Additive Trend"
))

fig.update_layout(
    title="Trend Component",
    xaxis_title="Date",
    yaxis_title="VWAP",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)

fig.show()

<div class="alert alert-block alert-info">

<a id="arima"></a>

### AUTO-REGRESSIVE INTEGRATED MOVING AVERAGE MODELS

<div class="alert alert-block alert-warning">

Auto-regressive (AR) and moving average (MA) models are popular models that are frequently used for forecasting.
AR and MA models are combined to create models such as auto-regressive moving average (ARMA) and auto-regressive
integrated moving average (ARIMA) models. 

ARMA models are basically regression models. 

Auto-regression simply means regression of a variable on itself measured at different time periods. We will discuss each component in the subsequent sections.

<div class="alert alert-block alert-info">

<a id="ar"></a>

### AR Models

<div class="alert alert-block alert-warning">

Auto-regression is a regression of a variable on itself measured at different time points. Auto-regressive model with lag 1,
AR (1), is given by


![](https://github.com/rakash/images1/blob/master/ar1.png?raw=true)

The above equation can be generalized to include p lags and is called a AR(p) model. The Equation can be re-written as;


![](https://github.com/rakash/images1/blob/master/ar2.png?raw=true)


where et+1 is a sequence of uncorrelated residuals assumed to follow the normal distribution with zero mean and constant standard deviation. 

(Yt − m) can be interpreted as a deviation from mean value m; it is known as mean centered series.

One of the important tasks in using the AR model in forecasting is model identification, which is, identifying the value of p (the number of lags). 

One of the standard approaches used for model identification is using
    - auto-correlation function (ACF) and 
    - partial auto-correlation function (PACF)

<div class="alert alert-block alert-info">

### ACF

<div class="alert alert-block alert-warning">

Auto-correlation of lag k is the correlation between Yt and Yt–k measured at different k values 
(e.g., Yt and Yt−1 or Yt and Yt−2). 

A plot of auto-correlation for different values of k is called an auto-correlation function 
(ACF) or correlogram. 

#### statsmodels.graphics.tsaplots.plot_acf plots the autocorrelation plot.

<div class="alert alert-block alert-info">

### PACF

<div class="alert alert-block alert-warning">

Partial auto-correlation of lag k is the correlation between Yt and Yt−k when the influence of all 
intermediate values (Yt−1, Yt−2, …, Yt−k+1) is removed (partial out) from both Yt and Yt−k. 

A plot of partial auto-correlation for different values of k is called partial auto-correlation function (PACF). 

statsmodels.graphics.tsaplots.plot_pacf plots the partial auto-correlation plot.

<div class="alert alert-block alert-info">

<a id="arma"></a>

### Building an ARMA MODEL

In [ ]:
#ACF plot to show auto-correlation upto lag of 20
    
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plt.rcParams.update({'figure.figsize': (10,6)})
acf_plot = plot_acf(vwap_itc_2020d.VWAP, lags=20)

The shaded area represents the upper and lower bounds for critical values, where the null hypothesis cannot be
rejected (auto-correlation value is 0). 

In [ ]:
plt.rcParams.update({'figure.figsize': (10,6)})

acf_plot = plot_pacf(vwap_itc_2020d.VWAP, lags=20)

<div class="alert alert-block alert-warning">

To select the appropriate p in the AR model, the following thumb rule can be used:

1. The partial auto-correlation is significant for first p-values (first p lags) and cuts off to zero.

2. The ACF decreases exponentially. (Signs of stationarity).

<div class="alert alert-block alert-info">

#### Building an AR Model

<div class="alert alert-block alert-warning">

statsmodels.tsa.arima_model.ARIMA can be used to build AR model.

It takes the following two parameters:

1. endog: list of values – It is the endogenous variable of the time series. 

2. order: The (p, d, q) – ARIMA model parameters. Order of AR is given by the value p, the order of integration is d, and the order of MA is given by q.

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
arima = ARIMA(vwap_itc_2020.VWAP[0:265].astype(np.float64), order = (1,0,0))
arima1 = ARIMA(vwap_itc_2020.VWAP[0:265].astype(np.float64), order = (0,0,0))
ar_model = arima.fit()
ar_model1 = arima1.fit()
ar_model.summary2()

<div class="alert alert-block alert-info">

### Forecasting and measuring accuracy

In [ ]:
forecast_arima = ar_model.predict(265, 283)
forecast_arima1 = ar_model1.predict(265, 283)

get_mape(vwap_itc_2020.VWAP[265:285].values, forecast_arima.values)

### A little higher than the moving average Mape value

### Forecast with P = 0 

In [ ]:
get_mape(vwap_itc_2020.VWAP[265:285].values, forecast_arima1.values)

<div class="alert alert-block alert-info">

<a id="ma"></a>

### Building an MA Model

<div class="alert alert-block alert-warning">

MA processes are regression models in which the past residuals are used for forecasting future values of the time-series data. 

A moving average process of lag 1 can be written as


![](https://github.com/rakash/images1/blob/master/ma1.png?raw=true)

The value of q (number of lags) in a moving average process can be identified using the following rules:

1. Auto-correlation value is significant for first q lags and cuts off to zero. 

2. The PACF decreases exponentially. 

Building a baseline with q=1

In [ ]:
arima = ARIMA(vwap_itc_2020.VWAP[0:265].astype(np.float64), order = (0,0,1))
ma_model = arima.fit()
ma_model.summary2()

In [ ]:
forecast_arima = ma_model.predict(265, 283)

get_mape(vwap_itc_2020.VWAP[265:285].values, forecast_arima.values)

### The mape is high

<div class="alert alert-block alert-info">

<a id="arma"></a>

### Building the ARMA Model

<div class="alert alert-block alert-warning">

Auto-regressive moving average (ARMA) is a combination auto-regressive and moving average process. 
ARMA(p, q) process combines AR(p) and MA(q) processes.
The values of p and q in an ARMA process can be identified using the following thumb rules:
    
1. Auto-correlation values are significant for first q values (first q lags) and cuts off to zero. 
2. Partial auto-correlation values are significant for first p values and cuts off to zero.

In [ ]:
arima = ARIMA(vwap_itc_2020.VWAP[0:265].astype(np.float64), order = (1,0,1)) 
arma_model = arima.fit() 
arma_model.summary2()

In [ ]:
forecast_arma = arma_model.predict(265, 283)
get_mape(vwap_itc_2020.VWAP[265:285].values, forecast_arma.values)

#### The Mape seems to be the same as AR(1) model. 

<div class="alert alert-block alert-info">

<a id="arimaa"></a>

### ARIMA Model

<div class="alert alert-block alert-warning">

ARMA models can be used only when the time-series data is stationary. 
ARIMA models are used when the time-series data is non-stationary. 
Time-series data is called stationary if the mean, variance, and covariance are constant over time. 


ARIMA has the following three components and is represented as ARIMA (p, d, q): 
1. AR component with p lags AR(p). 
2. Integration component (d). 
3. MA with q lags, MA(q).


The main objective of the integration component is to convert a non-stationary time-series process to a stationary process so that the AR and MA processes can be used for forecasting.

<div class="alert alert-block alert-info">

#### What is Stationary data?

Time-series data should satisfy the following conditions to be stationary: 
    
1. The mean values of Yt at different values of t are constant. 
2. The variances of Yt at different time periods are constant (Homoscedasticity). 
3. The covariance of Yt and Yt−k for different lags depend only on k and not on time t.

<div class="alert alert-block alert-info">
### Finding if a series is Stationary

<div class="alert alert-block alert-warning">

To find out if a time series is stationary, Dickey−Fuller test can also be conducted.

Dickey–Fuller test checks whether the b in the equation in AR model section above is equal to 1 or less than equal to 1. 

Equation: ![](https://github.com/rakash/images1/blob/master/ar1.png?raw=true)

It is a hypothesis test in which the null hypothesis and alternative hypothesis are given by,

H0: b = 1 (the time series is non-stationary)
HA: b < 1 (the time series is stationary)
    
statsmodels.tsa.stattools.adfuller is a Dicky−Fuller test and returns test statistics and p-value for the test of the null hypothesis.

If the p-value is less than 0.05, the time series is stationary.

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adfuller_test(ts): 
    adfuller_result = adfuller(ts, autolag=None)
    adfuller_out = pd.Series(adfuller_result[0:4],
                             index=['Test Statistic', 'p-value', 'Lags Used', 'Number of Observations Used'])
    print(adfuller_out)

In [ ]:
adfuller_test(vwap_itc_2020.VWAP)

<div class="alert alert-block alert-warning">

The p-value (>0.05) indicates that we cannot reject the null hypothesis and hence, the series is not stationary. 

Differencing the original time series is an usual approach for converting the non-stationary process into a stationary process (called difference stationarity). 

For example, the first difference (d = 1) is the difference between consecutive values of the time series (Yt). That is, the first difference is given by

![](https://github.com/rakash/images1/blob/master/dk1.png?raw=true)

<div class="alert alert-block alert-info">

### Differencing

<div class="alert alert-block alert-warning">

First difference between consecutive Yt values can be computed by subtracting the previous day’s value from that day’s value.

We can use shift() function in Pandas to shift the values before subtracting. 

In [ ]:
vwap_itc_2020['vdiff'] = vwap_itc_2020.VWAP - vwap_itc_2020.VWAP.shift(1)
vwap_itc_2020.head(5)
vwap_itc_2020 = vwap_itc_2020.dropna()

<div class="alert alert-block alert-info">

### Plotting the first-order difference values

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

vwap_itc_20201 = vwap_itc_2020.reset_index()

fig.add_trace(go.Scatter(
    x=vwap_itc_20201['Date'],
    y=vwap_itc_20201['vdiff'],
    name="First-Order Differencing"
))

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="First-order difference",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)

fig.show()

### The ACF plot for the same

In [ ]:
pacf_plot = plot_acf(vwap_itc_20201.vdiff.dropna(), lags=10)

<div class="alert alert-block alert-info">

### We build a basic arima model with d=1(first-order differencing)

In [ ]:
arima = ARIMA(vwap_itc_2020.VWAP[0:265].astype(np.float64), order = (1,1,1)) 
arima_model = arima.fit()
arima_model.summary2()

<div class="alert alert-block alert-info">
### Forecasting using the d component

In [ ]:
predict, stderr, ci = arima_model.forecast(steps=7)
get_mape(vwap_itc_2020.VWAP[265:285].values, predict)

<div class="alert alert-block alert-warning">
### So, The ARIMA model with first-order differencing gives forecast accuracy of ~10%.

<div class="alert alert-block alert-info">

<a id="summ"></a>

### Summary:

1. Several techniques such as moving average, exponential smoothing, and auto-regressive models are used for forecasting future value of Yt. 

2. The forecasting models are mostly validated using accuracy measures such as RMSE and MAPE. 

3. Auto-regressive (AR) models are regression-based models in which dependent variable is Yt and the independent variables are Yt−1, Yt−2, etc. 

4. AR models can be used only when the data is stationary.

5. Moving average (MA) models are regression models in which the independent variables are past error values. 

6. Auto-regressive integrated moving average (ARIMA) has three components: 
    a. Auto-regressive component with p lags − AR(p) 
    b. Moving average component with q lags − MA(q) 
    c. Integration which is differencing the original data to make it stationary (denoted by d)

<div class="alert alert-block alert-warning">

### End of Basics - Will add more approaches(ARIMAX/SARIMAX, NN's) to this notebook. Do upvote! 